In [8]:
from IPython.utils import io

try:
    with io.capture_output():
        %run -i "Infos sur les lieux des compétitions.ipynb"
except Exception as e:
    display(e)

In [9]:
from IPython.utils import io

try:
    with io.capture_output():
        %run -i "Nettoyage données affluence par jour + nombre de validation par jour par station.ipynb"
except Exception as e:
    display(e)

In [10]:
from IPython.utils import io

try:
    with io.capture_output():
        %run -i "Calendrier.ipynb"
except Exception as e:
    display(e)

In [13]:
def calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station):

    tableau_debut = pd.DataFrame()  # Initialisation du DataFrame vide
    tableau_fin = pd.DataFrame()    # Initialisation du DataFrame vide

    cle_tableau = f'ligne numéro[{numero_ligne}]'

    # Extraire l'horaire de l'entrée utilisateur
    heure_entree = date_horaire_entree[-5:]

    # Déterminer la plage horaire à laquelle appartient l'horaire d'entrée
    plage_horaire_entree = f"{heure_entree.split(':')[0]}:00:00 - {(int(heure_entree.split(':')[0])+1)%24:02d}:00:00"

    # Construire la chaîne de date avec la plage horaire
    date_plage_horaire = f"{date_horaire_entree[:10]} {plage_horaire_entree}"

    # Vérifier si la chaîne date_plage_horaire se trouve dans new_horaires_début_df
    if (date_plage_horaire in new_horaires_début_df.values) or (date_plage_horaire in new_horaires_fin_df.values): 
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_début_df
        lieux_correspondants_debut = new_horaires_début_df.loc[new_horaires_début_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_fin_df
        lieux_correspondants_fin = new_horaires_fin_df.loc[new_horaires_fin_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        
        lignes_correspondantes = []
        
        
        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_debut
        for lieu in lieux_correspondants_debut:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_debut=pd.concat([tableau_debut, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)
            

        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_fin
        for lieu in lieux_correspondants_fin:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_fin=pd.concat([tableau_fin, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)

        numero_trouve = False

        for sous_liste in lignes_correspondantes:
            if numero_ligne in sous_liste:
                numero_trouve = True
                break  # Arrête la boucle une fois que le numéro est trouvé

            # Vérifier si le numéro de ligne entré est présent dans les vecteurs des lignes correspondantes
        if numero_trouve:
            resultat_debut=0
            resultat_fin=0
            if not tableau_debut.empty :
                tableau_debut = tableau_debut[tableau_debut['Lignes'].apply(lambda x: numero_ligne in x)]
                resultat_debut = tableau_debut['Nombre de personnes par ligne'].sum()
            if not tableau_fin.empty :
                    tableau_fin = tableau_fin[tableau_fin['Lignes'].apply(lambda x: numero_ligne in x)]
                    resultat_fin = tableau_fin['Nombre de personnes par ligne'].sum()
            resultat=resultat_debut+resultat_fin
            nb_ligne=round(resultat)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré  sur la ligne en raison des JO  est {nb_ligne}.")

            resultat_debut_2=0
            resultat_fin_2=0

            if not tableau_debut.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_debut['Pourcentage fréquentation'] = valeur
                resultat_debut_2=valeur*resultat_debut
            
            if not tableau_fin.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_fin['Pourcentage fréquentation'] = valeur
                resultat_fin_2=valeur*resultat_debut
            
            resultat2=resultat_debut_2+resultat_fin_2
            nb_station=round(resultat2)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré à la station en raison des JO  est {nb_station}.")

        else:
            print(0)
    else:
        print(0)

In [14]:
calculer_personnes_supplementaires('2024-07-28 09:20', '1', 'bastille')

KeyError: 'Lieu'

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)